In [ ]:
from keras.datasets import imdb
from keras import preprocessing

In [8]:
from google.colab import drive
drive.mount('/content/sample_data/Summary')

Mounted at /content/sample_data/Summary


In [9]:
%pwd


[Errno 2] No such file or directory: '(/content/sample_data/)'
/content


In [1]:
from google.colab import files
upload = files.upload()

Saving 001.txt to 001.txt
Saving 002.txt to 002.txt
Saving 003.txt to 003.txt
Saving 004.txt to 004.txt
Saving 005.txt to 005.txt
Saving 006.txt to 006.txt
Saving 007.txt to 007.txt
Saving 008.txt to 008.txt
Saving 009.txt to 009.txt
Saving 010.txt to 010.txt
Saving 011.txt to 011.txt
Saving 012.txt to 012.txt
Saving 013.txt to 013.txt
Saving 014.txt to 014.txt
Saving 015.txt to 015.txt
Saving 016.txt to 016.txt
Saving 017.txt to 017.txt
Saving 018.txt to 018.txt
Saving 019.txt to 019.txt
Saving 020.txt to 020.txt
Saving 021.txt to 021.txt
Saving 022.txt to 022.txt
Saving 023.txt to 023.txt
Saving 024.txt to 024.txt
Saving 025.txt to 025.txt
Saving 026.txt to 026.txt
Saving 027.txt to 027.txt
Saving 028.txt to 028.txt
Saving 029.txt to 029.txt
Saving 030.txt to 030.txt
Saving 031.txt to 031.txt
Saving 032.txt to 032.txt
Saving 033.txt to 033.txt
Saving 034.txt to 034.txt
Saving 035.txt to 035.txt
Saving 036.txt to 036.txt
Saving 037.txt to 037.txt
Saving 038.txt to 038.txt
Saving 039.t

In [ ]:
max_features = 10000
maxlen = 20

In [ ]:
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_features)

17465344/17464789 [==============================] - 0s 0us/step


In [ ]:
x_train = preprocessing.sequence.pad_sequences(x_train,maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test,maxlen=maxlen)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten , Dense , Embedding

In [ ]:
model = Sequential()
model.add(Embedding(10000, 8 ,input_length=maxlen))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train,y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 1s 2ms/step - loss: 0.6649 - acc: 0.6270 - val_loss: 0.6085 - val_acc: 0.6978
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 0.5301 - acc: 0.7513 - val_loss: 0.5204 - val_acc: 0.7316
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4576 - acc: 0.7885 - val_loss: 0.4981 - val_acc: 0.7456
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4232 - acc: 0.8050 - val_loss: 0.4923 - val_acc: 0.7550
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4011 - acc: 0.8192 - val_loss: 0.4932 - val_acc: 0.7552
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3824 - acc: 0.8304 - val_loss: 0.4968 - val_acc: 0.7582
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3663 - acc: 0.8395 - val_loss: 0.4994 - val_acc: 0.7614
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0.